## Imports and Data

### Download Dataset First (If not previously downloaded)

In [24]:
# !wget https://hkustconnect-my.sharepoint.com/:u:/g/personal/nnanda_connect_ust_hk/ESUV7uEth0NEvr_r022vxxEBj111I6yEQSIHREQnYX3K5A?download=1 -O new_data.zip
# !unzip -q -o new_data.zip

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

england_matches = pd.read_json('new_data/matches/matches_England.json')
teams = pd.read_json('new_data/teams.json')
england_events = pd.read_json('new_data/events/events_England.json')
players = pd.read_json('new_data/players.json')

In [26]:
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
from IPython.display import IFrame
import json

## Team Analysis
### Team class for accessing team attributes (id, matches, players, events etc.) 


In [27]:
class team:
    def __init__(self, name:str):
        self.name = name
        self.team_id = self.get_team_id()
        self.matches = self.get_matches()
        self.players = self.get_players()
        self.all_events = self.get_all_events()
    
    def get_team_id(self):
        return int(teams[teams['name'] == self.name]['wyId'])
        
    def get_matches(self, opposition_team=None):
        matches = pd.DataFrame(columns=england_matches.columns)
        specific_matches = pd.DataFrame(columns=england_matches.columns)
        
        for index,row in england_matches.iterrows():
            teams_in_match = [int(team) for team in list(row['teamsData'].keys())]
            if opposition_team is None:
                if self.team_id in teams_in_match:
                    matches = matches.append(row)
            else:
                if self.team_id in teams_in_match and opposition_team.team_id in teams_in_match:
                    specific_matches = specific_matches.append(row)
                
        if opposition_team is None:     
            return matches
        else:
            return specific_matches
    
    def get_players(self):
        return players[players['currentTeamId'] == self.team_id]
    
    def get_players_in_match(self, match_id:int):
        target_match_events = self.all_events[self.all_events['matchId'] == match_id]
        target_match_player_ids = list(set(target_match_events['playerId'].tolist()))
        target_match_players = self.players[self.players['wyId'].isin(target_match_player_ids)]
        return target_match_players
    
    def get_all_events(self):
        return england_events[england_events['teamId'] == self.team_id]


## Graphics Generation

In [28]:
from matplotlib.patches import Arc

kde_plot = None

def draw_pitch(match=None, team=None, player=None, event=None):
    global ax
    
    fig = plt.figure()
    fig.set_size_inches(14,7)

    ax = plt.subplot(111)
    
    ax.set_title('{} \n {} - {} - {}'.format(match, team, player, event), fontsize=20)
    
    #touchline
    plt.plot([0,0], [-5,105], 'k') #left length
    plt.plot([0,100], [105,105], 'k') #upper width
    plt.plot([100,100], [-5,105], 'k') #right length
    plt.plot([100,0], [-5,-5], 'k') #lower width
    plt.plot([50,50], [-5,105], 'k') #halfway line

    #center
    center_circle = plt.Circle((50,50), radius=7, color='black', fill=False)
    center_spot = plt.Circle((50,50), radius=0.7, color='black', fill=True)
    ax.add_artist(center_circle)
    ax.add_artist(center_spot)

    #left 18 yard box
    plt.plot([0,12], [32,32], 'k')
    plt.plot([0,12], [69,69], 'k')
    plt.plot([12,12], [32,69], 'k')
    left_d = Arc((6, 50), width=18.3, height=20, theta1=310, theta2=50, color='black')
    ax.add_patch(left_d)

    #left 6 yard box
    plt.plot([0,4], [41.25,41.25], 'k')
    plt.plot([0,4], [59.75,59.75], 'k')
    plt.plot([4,4], [59.75,41.25], 'k')

    #right 18 yard box
    plt.plot([100,88], [32,32], 'k')
    plt.plot([100,88], [69,69], 'k')
    plt.plot([88,88], [32,69], 'k')
    right_d = Arc((94, 50), width=18.3, height=20, theta1=130, theta2=230, color='black')
    ax.add_patch(right_d)

    #right 6 yard box
    plt.plot([100,96], [41.25,41.25], 'k')
    plt.plot([100,96], [59.75,59.75], 'k')
    plt.plot([96,96], [59.75,41.25], 'k')
    
    return fig, ax

#dataframe should be an 'events' dataframe
def draw_heatmap(dataframe):
    global kde_plot
    kde_plot = sns.kdeplot(dataframe['x_start'], dataframe['y_start'], clip=((0,100), (-5,105)), shade=True, shade_lowest=False, cmap='Greens', n_levels = 20)
    plt.axis('off')

def draw_scatter(dataframe):
    global kde_plot
    sns.scatterplot(dataframe['x_start'], dataframe['y_start'])
    plt.axis('off')
    plt.show()

# plt.show()

## Dashboard

In [29]:
from ipywidgets import interact, interactive, interact_manual, widgets, Layout
import warnings 
warnings.filterwarnings('ignore')

def heatmap_dashboard():
    #global variables whose values will be updated with every successive selection : filtering achieved through modification of these global vars
    team_1, team_2 = None, None
    matches = None
    target_match = None
    target_team = None
    target_player = None
    events = None
    selected_events = None
    target_events = None

    #First selector that user sees is team selector : get its options
    england_teams = pd.DataFrame(columns=teams.columns)
    for index,row in teams.iterrows():
        if row['area']['name'] == 'England':
            england_teams = england_teams.append(row)
    team_selector_options = sorted(england_teams['name'].tolist())

    #create team selectors - first interactive element to be called, invocation is on last line 
    team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
    team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

    def select_teams(team_1_selector, team_2_selector):
        global team_1, team_2, matches

        team_1 = team(team_1_selector)
        team_2 = team(team_2_selector)
        matches = team_1.get_matches(team_2)

        #teams selected, call match selector to select one of 2 matches between them
        match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match:', layout=Layout(margin='0px 0px 0px -3px'))
        interact(select_match, match_selector=match_selector)

    def select_match(match_selector) -> pd.DataFrame:
        global matches, target_match, team_1, team_2

        target_match = matches[matches['label'] == match_selector]
#         display(target_match) #for debugging
    
        #match selected, select which team in match to analyse
        target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team:', layout=Layout(margin='0px 0px 0px -8px'))
        interact(select_team, team_selector=target_team_selector)

    def select_team(team_selector):
        global team_1, team_2, target_match, target_team, events

        target_team = team_1 if team_selector == team_1.name else team_2

        events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])] 
        
        #unwrap xy coords of events : originally stored as dictionary elements in column 'positions'
        events['x_start'] = events.apply(lambda row: row['positions'][0]['x'], axis=1)
        events['y_start'] = events.apply(lambda row: 100-row['positions'][0]['y'], axis=1)
        
        #event times measured from start of every half => have to check which period an event is in : avoid by measuring all events from start of first half
        events.loc[events.matchPeriod == '2H', 'eventSec'] = 45*60 + events['eventSec']
        
        #target team selected, now select (or not) target player
        players_in_target_match = target_team.get_players_in_match(match_id=int(target_match['wyId']))
        player_options = sorted(players_in_target_match['shortName'].tolist())
        player_options.insert(0, 'All Players')
        
        player_selector = widgets.Dropdown(options=player_options, description='Player:', layout=Layout(margin='0px 0px 0px -14px'))
        interact(select_player, player_selector=player_selector)

    def select_player(player_selector):
        global target_player, target_team, events
        
        target_player_name = player_selector
                                
        #filter events only if a player is selected
        if target_player_name != 'All Players':
            target_player = target_team.players[target_team.players['shortName'] == target_player_name]
        else:
            target_player = None
            
        #target team selected, now select type of event to analyse
        event_selector = widgets.Dropdown(options=sorted(set(events['eventName'].tolist())), description='Event:', layout=Layout(margin='0px 0px 0px -20px'))
        interact(select_event, event_selector=event_selector)

        
    def select_event(event_selector):
        global target_team, target_player, events, selected_events
        
        if target_player is None:
            selected_events = events[events['eventName'] == event_selector]
        else:
            target_player_id = int(target_player['wyId'])
            target_player_events = events[events['playerId'] == target_player_id]
            selected_events = target_player_events[target_player_events['eventName'] == event_selector]
        
        #type of event selected, now select time interval 
        range_selector = widgets.IntRangeSlider(value=[0, 90], min=0, max=90, step=10,description='Interval:',disabled=False, layout=Layout(margin='0px 0px 0px 1px'))
        interact(select_range, range_selector=range_selector)

    def select_range(range_selector):
        global selected_events, target_match, target_team, target_player, target_events

        #target events is selected_events constrained to a particular interval
        target_events = selected_events[selected_events['eventSec'] >= range_selector[0] * 60]
        target_events = target_events[target_events['eventSec'] <= range_selector[1] * 60]
        
#         print(target_events.shape) #for debugging

        #======= EVERYTHING SELECTED, LEZZGO ==========#
        plot_individual_events = widgets.Checkbox(value=False,description=': Plot individual events', indent=True, layout=Layout(margin='0px 0px 0px -20px'))
        interact(plot, plot_individual_events=plot_individual_events)
        
        
    def plot(plot_individual_events):
        global selected_events, target_match, target_team, target_player, target_events
        
        #find focus of attacks based on y coord
        right_wing_events = target_events[target_events['y_start'] <= 33]
        left_wing_events = target_events[target_events['y_start'] >= 67]
        midfield_events = target_events[(target_events['y_start'] > 33) & (target_events['y_start'] < 67)]
        
        right_focus = str(int(right_wing_events.shape[0]/target_events.shape[0]*100)) + '%'
        left_focus = str(int(left_wing_events.shape[0]/target_events.shape[0]*100)) + '%'
        middle_focus = str(int(midfield_events.shape[0]/target_events.shape[0]*100)) + '%'
        
        # set plot title : get target player name, else write 'All Players'
        if isinstance(target_player, pd.DataFrame):
            player_name = target_player.iloc[0]['firstName'] + ' ' + target_player.iloc[0]['lastName']
        else:
            player_name = 'All Players'
            
        draw_pitch(target_match.iloc[0]['label'], target_team.name, player_name, "{} \n Right:{} | Left:{} | Middle: {}".format(target_events.iloc[0]['eventName'], right_focus, left_focus, middle_focus))
        #prevent heatmap generation for sparse dataframes
        try:
            draw_heatmap(target_events)
            if plot_individual_events:
                draw_scatter(target_events)
#                 plt.show()
        except Exception as e:
            print(e) #for debugging
            print("Not Enough Events For Density Estimation")
            
        plt.show()

    #invoke team selector : get the ball rolling
    interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

In [30]:
heatmap_dashboard()

interactive(children=(Dropdown(description='Team 1:', options=('AFC Bournemouth', 'Arsenal', 'Brighton & Hove …

## Build-Up to Goal Map

In [31]:
import math

def buildup_dashboard():
    #global variables whose values will be updated with every successive selection : filtering achieved through modification of these global vars
    team_1, team_2 = None, None
    matches = None
    target_match = None
    target_team = None
#     target_player = None
    events = None
    goals = None
    target_goal_index = None
    selected_events = None
    target_events = None

    #First selector that user sees is team selector : get its options
    england_teams = pd.DataFrame(columns=teams.columns)
    for index,row in teams.iterrows():
        if row['area']['name'] == 'England':
            england_teams = england_teams.append(row)
    team_selector_options = sorted(england_teams['name'].tolist())

    #create team selectors - first interactive element to be called, invocation is on last line 
    team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
    team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

    def select_teams(team_1_selector, team_2_selector):
        global team_1, team_2, matches

        team_1 = team(team_1_selector)
        team_2 = team(team_2_selector)
        matches = team_1.get_matches(team_2)

        #teams selected, call match selector to select one of 2 matches between them
        match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match:', layout=Layout(margin='0px 0px 0px -3px'))
        interact(select_match, match_selector=match_selector)

    def select_match(match_selector) -> pd.DataFrame:
        global matches, target_match, team_1, team_2

        target_match = matches[matches['label'] == match_selector]
#         display(target_match) #for debugging
    
        #match selected, select which team in match to analyse
        target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team:', layout=Layout(margin='0px 0px 0px -8px'))
        interact(select_team, team_selector=target_team_selector)

    def select_team(team_selector):
        global team_1, team_2, target_match, target_team, events, goals

        target_team = team_1 if team_selector == team_1.name else team_2

        events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])] 
        
        #unwrap xy coords of events : originally stored as dictionary elements in column 'positions'
        events['x_start'] = events.apply(lambda row: row['positions'][0]['x'], axis=1)
        events['y_start'] = events.apply(lambda row: 100-row['positions'][0]['y'], axis=1)
        
        #event times measured from start of every half => have to check which period an event is in : avoid by measuring all events from start of first half
        events.loc[events.matchPeriod == '2H', 'eventSec'] = 45*60 + events['eventSec']
        
        #target team selected, now select target goal
        shots = events[events['eventName'] == 'Shot']

        goals = pd.DataFrame(columns=shots.columns)
        goal_indices = []

        for index,row in shots.iterrows():
            tags = [pair['id'] for pair in row['tags']]
            if 101 in tags:
                goals = goals.append(row)
                goal_indices.append(index)
        
        goals['goal_minute'] = goals.apply(lambda row:int(row['eventSec']/60), axis=1)
        # display(afc_lei_goals)
#         display(goal_indices)
#         display(goals)
        
        goal_selector = widgets.Dropdown(options=goals['goal_minute'].tolist(), description='Goal:', layout=Layout(margin='0px 0px 0px -14px'))
        interact(select_goal, goal_selector=goal_selector)


    def select_goal(goal_selector):
        global target_player, target_team, events, goals, target_goal_index, target_match
        
        target_goal_minute = goal_selector
                                
        target_goal_index = goals.index[goals['goal_minute'] == int(target_goal_minute)][0]
#         print("INDEX: ", target_goal_index)
            
        events_leading_to_goal = events.loc[target_goal_index-7:target_goal_index]
        
        coords = list(zip(events_leading_to_goal['x_start'], events_leading_to_goal['y_start']))
        
        total_pass_length = 0
        for index,tup in enumerate(coords[:len(coords)-1]):
            total_pass_length+= math.sqrt((coords[index+1][0]-tup[0])**2 + (coords[index+1][1]-tup[1])**2)
        avg_pass_length = int(total_pass_length/(len(coords)-1))

        fig, ax = draw_pitch(match=target_match.iloc[0]['label'], team=target_team.name, player='Goal', event='Minute {} \n Avg. Pass Length - {} yards'.format(target_goal_minute, avg_pass_length))
#         ax.set_facecolor('green')
        scatter = ax.scatter(x=events_leading_to_goal['x_start'], y=events_leading_to_goal['y_start'])

        
        for index,tup in enumerate(coords[:len(coords)-1]):
            plt.arrow(x=tup[0], y=tup[1], dx=coords[index+1][0]-tup[0], dy=coords[index+1][1]-tup[1], shape='full', head_length=2., head_width=2., length_includes_head=True, color='b')
        
        plt.arrow(x=coords[-1][0], y=coords[-1][1], dx=100-coords[-1][0], dy=50-coords[-1][1], color='r', head_length=2, head_width=2, length_includes_head=True)
        plt.axis('off')
        plt.show()

    #invoke team selector : get the ball rolling
    interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

In [32]:
buildup_dashboard()

interactive(children=(Dropdown(description='Team 1:', options=('AFC Bournemouth', 'Arsenal', 'Brighton & Hove …

In [33]:
from ipywidgets import interact, interactive, interact_manual, widgets, Layout
import warnings 
warnings.filterwarnings('ignore')

def graph_dashboard():
    #global variables whose values will be updated with every successive selection : filtering achieved through modification of these global vars
    team_1, team_2 = None, None
    matches = None
    target_match = None
    target_team = None
    target_player = None
    events = None
    selected_events = None
    teamData = None
    target_match_not = None

    #First selector that user sees is team selector : get its options
    england_teams = pd.DataFrame(columns=teams.columns)
    for index,row in teams.iterrows():
        if row['area']['name'] == 'England':
            england_teams = england_teams.append(row)
    team_selector_options = sorted(england_teams['name'].tolist())

    #create team selectors - first interactive element to be called, invocation is on last line 
    team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
    team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

    def select_teams(team_1_selector, team_2_selector):
        global team_1, team_2, matches

        team_1 = team(team_1_selector)
        team_2 = team(team_2_selector)
        matches = team_1.get_matches(team_2)

        #teams selected, call match selector to select one of 2 matches between them
        match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match:', layout=Layout(margin='0px 0px 0px -3px'))
        interact(select_match, match_selector=match_selector)

    def select_match(match_selector) -> pd.DataFrame:
        global matches, target_match, team_1, team_2, target_match_not

        target_match = matches[matches['label'] == match_selector]
        target_match_not = matches[matches['label'] != match_selector]
#         display(target_match) #for debugging
    
        #match selected, select which team in match to analyse
        target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team:', layout=Layout(margin='0px 0px 0px -8px'))
        interact(select_team, team_selector=target_team_selector)
        
    def select_team(team_selector):
        global team_1, team_2, target_match, target_team, events, target_match_not

        target_team = team_1 if team_selector == team_1.name else team_2

        events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])] 
        events_not = target_team.all_events[target_team.all_events['matchId'] == int(target_match_not['wyId'])]
        print(target_match.iloc[0]['label'])
        print(target_match_not.iloc[0]['label'])

#         print(events.shape)
#         print(events_not.shape)
        
#         print(events.head())
#         teamId = events['teamId'][0:1]
        teamId = events.iloc[0]['teamId']
#         print('Team ID', teamId)
#         print(target_match.iloc[0]['teamsData'])
        lineup = target_match.iloc[0]['teamsData']
#         print(lineup[str(teamId)]['formation']['lineup'])
        first_11 = lineup[str(teamId)]['formation']['lineup']
    
#         first_11 = team_info['1609']['formation']['lineup']
        players_id = []
        players_dict = {}
        players_dict_ulta = {}
        for i in range(len(first_11)):
        #     print(first_11[i]['playerId'])
            players_id.append(first_11[i]['playerId'])
            players_dict[first_11[i]['playerId']] = i
            players_dict_ulta[i] = first_11[i]['playerId']

#         print(players_id)
#         print(players_dict)
#         print(players_dict_ulta)

        players_names = {}
        for i in range(11):
            pid = players_dict_ulta[i]
            players_names[i] = players[players['wyId']==pid].iloc[0]['shortName']

#         print(players_names)
        
        team_pass = events[events['eventName']=='Pass']
        team_pass = team_pass[team_pass['playerId'].isin(players_id)]
        team_pass['Length'] = team_pass.tags.apply(lambda x: len(x))
        team_pass = team_pass[team_pass['Length']==1]
        team_pass['accurate'] = team_pass.apply(lambda row: row['tags'][0]['id'], axis=1)
        team_pass = team_pass[team_pass['accurate']==1801]
        team_pass = team_pass.reset_index(drop=True)
#         print(team_pass.head())
        
        players_matrix = np.zeros((11, 11), dtype=int)
        
        for i in range(1, len(team_pass)):
            #print(arsenal_pass.loc[i, 'eventSec']-arsenal_pass.loc[i-1, 'eventSec'])
            if team_pass.loc[i, 'eventSec']-team_pass.loc[i-1, 'eventSec'] < 5.0:
                #print(arsenal_pass.loc[i, 'eventSec']-arsenal_pass.loc[i-1, 'eventSec'])
                player_from = team_pass.loc[i-1, 'playerId']
                player_to = team_pass.loc[i, 'playerId']
                #print(player_from, player_to)
#                 print(players_dict[player_from], players_dict[player_to])
                if player_from != player_to:
                    players_matrix[players_dict[player_from]][players_dict[player_to]] += 1
                    players_matrix[players_dict[player_to]][players_dict[player_from]] += 1
                    
#         print(players_matrix)
        
        G = nx.from_numpy_matrix(players_matrix)
        
        G = nx.relabel_nodes(G, players_names)
        
#         print(G.nodes())
        
#         nx.draw(G, with_labels=True)   #Does not show anything
        
#         print(list(G.edges(data=True)))

        cent = nx.algorithms.centrality.betweenness_centrality(G, weight='weight')
#         cent = nx.algorithms.centrality.betweenness_centrality(G)
#         print(cent)
    
        req = sorted(cent.items(), key=lambda x:x[1])
#         print('Top 3 players Betweenness:')
#         print(req[-1])
#         print(req[-2])
#         print(req[-3])
        
        nx.set_node_attributes(G, cent, 'betweenness')
        
        # For Group
        grp = {}
        count = 0
        for p in req:
            if count>7:
                name, _ = p
                grp[name] = 1
            else:
                name, _ = p
                grp[name] = 0
                
            count+=1
                
#         print(grp)
        
        nx.set_node_attributes(G, grp, 'group')
        
        clus = nx.clustering(G, weight='weight')
#         print(clus)
        
        nx.set_node_attributes(G, clus, 'Cluster')
        
#         print(G.nodes(data=True))
        
        team_json = json_graph.node_link_data(G)
#         print(team_json)
        
        with open("sample_weight_clus.json", "w") as outfile: 
            json.dump(team_json, outfile)
            
#         display(IFrame(src='./d3_tip.html', width=900, height=600))
#         display(IFrame(src='./d3_tip.html', width=900, height=600))

        display("Player Graph Successfully Generated at ./d3_tip.html")



    #invoke team selector : get the ball rolling
    interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

In [34]:
from IPython.display import clear_output

heatmap_dashboard_active = False
buildup_dashboard_active = False
graph_dashboard_active = False

def master_dashboard():
    heatmap_dashboard_button = widgets.Button(description="Team Analysis")
    heatmap_dashboard_button.on_click(display_heatmap_dashboard)
    display(heatmap_dashboard_button)
    
    buildup_dashboard_button = widgets.Button(description="Goal Buildup")
    buildup_dashboard_button.on_click(display_buildup_dashboard)
    display(buildup_dashboard_button)
    
    graph_dashboard_button = widgets.Button(description="Player Dynamics")
    graph_dashboard_button.on_click(display_graph_dashboard)
    display(graph_dashboard_button)

def display_heatmap_dashboard(b):
    global heatmap_dashboard_active
    if not heatmap_dashboard_active:
        heatmap_dashboard_active = True
        heatmap_dashboard()
    else:
        heatmap_dashboard_active = False
        clear_output()
        master_dashboard()
    
def display_buildup_dashboard(b):
    global buildup_dashboard_active
    if not buildup_dashboard_active:
        buildup_dashboard_active = True
        buildup_dashboard()
    else:
        buildup_dashboard_active = False
        clear_output()
        master_dashboard()
        
def display_graph_dashboard(b):
    global graph_dashboard_active
    if not graph_dashboard_active:
        graph_dashboard_active = True
        graph_dashboard()
    else:
        graph_dashboard_active = False
        clear_output()
        master_dashboard()
        
# master_dashboard()

In [35]:
master_dashboard()

Button(description='Team Analysis', style=ButtonStyle())

Button(description='Goal Buildup', style=ButtonStyle())

Button(description='Player Dynamics', style=ButtonStyle())

interactive(children=(Dropdown(description='Team 1:', options=('AFC Bournemouth', 'Arsenal', 'Brighton & Hove …

In this project, we present SoccerMetrics : an interactive data visualization system for analysing football team at an individual level (intra-team and player-player dynamics) and a collective level (opponent-dependent strategies and tactics). 

In SoccerMetics, we first use a player view to quantify the importance of a player to his team. This is done using a player-player pass graph to encode player dynamics, dynamic heatmaps of events to represent his dominance on the pitch, and control heatmaps that show how a team's performance suffers in his absence. Performing this importance analysis for 'n' players allows us to define the team's spine : the set of 'n' players most crucial to the team's success.

Once the important players of a team have been identified, we then use SoccerMetrics' team view to analyse the different strategies adopted by a team, given the calibre of the opponent. Here, team-level event heatmaps are used to encode the locations and degree of a team's dominance in a particular match, build-up play passing networks are used to illustrate pass length and frequency so as to identify the team's chance-creation strategies, and parallel coordinates are used to reinforce the conclusions drawn from both these visualizations. This analysis allows us to see whether or not a team changes its attacking, defending and tactical strategies depending on the quality of its opposition.

Finally, having identified which players and which strategies a team uses to be successful, we visualise and analyse an outlier match: a match which the team lost. In this visualisation, we look at the opposition's (the winner's) strategies to see how they disrupted the successful team's strategies. This could be in terms of nullifying the successful team's spine (once again visualised by the player view), or by interrupting and breaking-up its build-up play (using the team view).

Through this workflow, SoccerMetrics aims to analyse what players a team relies on, what strategies make it successful, and how to nullify these players and tactics in order to win.